<a href="https://colab.research.google.com/github/alexhenriquepineda/Previsao-Notas-ENEM/blob/main/ciclo-02-EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

In [3]:
zipfile.ZipFile('/content/drive/MyDrive/TCC/datasets/enem.zip', 'r').extractall('/content/drive/MyDrive/TCC/datasets')

**0.0 IMPORTS**

In [3]:
from tqdm import tqdm
import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from IPython.core.display  import HTML

**0.1 HELPER FUNCTION**

In [2]:
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    fonte: https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline  
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in tqdm([x for x in df.columns if 'NU_NOTA_' not in x]):
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [16, 8]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    
jupyter_settings()

pd.set_option('display.float_format', lambda x: '%.2f' % x)

Populating the interactive namespace from numpy and matplotlib


**1.0 LOADING DATA**

In [5]:
data_raw = pd.read_csv('/content/drive/MyDrive/TCC/datasets/train.csv')

In [6]:
data = reduce_mem_usage(data_raw)

Memory usage of dataframe is 2981.62 MB


100%|██████████| 113/113 [01:09<00:00,  1.63it/s]

Memory usage after optimization is: 732.86 MB
Decreased by 75.4%


**2.0 DATA DESCRIPTION**

In [7]:
data.columns = data.columns.str.lower()

In [8]:
#FONTE: DICIONÁRIO DE DADOS

dados_participante = ['nu_inscricao', 'co_municipio_residencia', 'no_municipio_residencia', 'co_uf_residencia', 'sg_uf_residencia', 'nu_idade', 'tp_sexo', 'tp_estado_civil', 'tp_cor_raca',
                      'tp_nacionalidade', 'co_municipio_nascimento', 'no_municipio_nascimento', 'co_uf_nascimento', 'sg_uf_nascimento', 'tp_st_conclusao', 'tp_ano_concluiu', 'tp_escola',
                      'tp_ensino', 'in_treineiro']

dados_escola = ['co_escola', 'co_municipio_esc', 'no_municipio_esc', 'co_uf_esc', 'sg_uf_esc', 'tp_dependencia_adm_esc', 'tp_localizacao_esc', 'tp_sit_func_esc']

dados_atendimento = ['in_baixa_visao', 'in_cegueira', 'in_surdez', 'in_deficiencia_auditiva', 'in_surdo_cegueira', 'in_deficiencia_fisica', 'in_deficiencia_mental', 'in_deficit_atencao',
                     'in_dislexia', 'in_discalculia', 'in_autismo', 'in_visao_monocular', 'in_outra_def', 'in_gestante', 'in_lactante', 'in_idoso', 'in_estuda_classe_hospitalar',
                     'in_sem_recurso', 'in_braille', 'in_ampliada_24', 'in_ampliada_18', 'in_ledor',  'in_acesso', 'in_transcricao', 'in_libras', 'in_tempo_adicional', 'in_leitura_labial',
                     'in_mesa_cadeira_rodas', 'in_mesa_cadeira_separada', 'in_apoio_perna', 'in_guia_interprete', 'in_computador', 'in_cadeira_especial', 'in_cadeira_canhoto', 
                     'in_cadeira_acolchoada', 'in_prova_deitado', 'in_mobiliario_obeso', 'in_lamina_overlay', 'in_protetor_auricular', 'in_medidor_glicose', 'in_maquina_braile', 'in_soroban',
                    'in_marca_passo', 'in_sonda', 'in_medicamentos', 'in_sala_individual', 'in_sala_especial', 'in_sala_acompanhante', 'in_mobiliario_especifico', 'in_material_especifico',
                     'in_nome_social']

dados_aplicacao_prova = ['co_municipio_prova', 'no_municipio_prova', 'co_uf_prova', 'sg_uf_prova']

dados_prova = ['tp_presenca_cn', 'tp_presenca_ch', 'tp_presenca_lc', 'tp_presenca_mt', 'nu_nota_cn', 'nu_nota_ch', 'nu_nota_lc', 'nu_nota_mt', 'tp_lingua', 'tp_status_redacao', 'nu_nota_redacao']

dados_socioeconomico = ['q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009', 'q010', 'q011', 'q012', 'q013', 'q014', 'q015', 'q016', 'q017', 'q018', 'q019', 'q020','q021',
                        'q022', 'q023', 'q024', 'q025']

**2.1 TAMANHO DA BASE**

In [9]:
data.shape

(3311925, 118)

**2.2 CHECK NA**

In [10]:
data[dados_participante].isna().sum()

nu_inscricao                     0
co_municipio_residencia          0
no_municipio_residencia          0
co_uf_residencia                 0
sg_uf_residencia                 0
nu_idade                        34
tp_sexo                          0
tp_estado_civil                  0
tp_cor_raca                      0
tp_nacionalidade                 0
co_municipio_nascimento      94562
no_municipio_nascimento      94562
co_uf_nascimento             94562
sg_uf_nascimento             94562
tp_st_conclusao                  0
tp_ano_concluiu                  0
tp_escola                        0
tp_ensino                  1441571
in_treineiro                     0
dtype: int64

In [11]:
data[dados_escola].isna().sum()

co_escola                 2566547
co_municipio_esc          2566547
no_municipio_esc          2566547
co_uf_esc                 2566547
sg_uf_esc                 2566547
tp_dependencia_adm_esc    2566547
tp_localizacao_esc        2566547
tp_sit_func_esc           2566547
dtype: int64

In [12]:
data[dados_atendimento].isna().sum()


in_baixa_visao                 0
in_cegueira                    0
in_surdez                      0
in_deficiencia_auditiva        0
in_surdo_cegueira              0
in_deficiencia_fisica          0
in_deficiencia_mental          0
in_deficit_atencao             0
in_dislexia                    0
in_discalculia                 0
in_autismo                     0
in_visao_monocular             0
in_outra_def                   0
in_gestante                    0
in_lactante                    0
in_idoso                       0
in_estuda_classe_hospitalar    0
in_sem_recurso                 0
in_braille                     0
in_ampliada_24                 0
in_ampliada_18                 0
in_ledor                       0
in_acesso                      0
in_transcricao                 0
in_libras                      0
in_tempo_adicional             0
in_leitura_labial              0
in_mesa_cadeira_rodas          0
in_mesa_cadeira_separada       0
in_apoio_perna                 0
in_guia_in

In [13]:
data[dados_aplicacao_prova].isna().sum()

co_municipio_prova    0
no_municipio_prova    0
co_uf_prova           0
sg_uf_prova           0
dtype: int64

In [14]:
data[dados_prova].isna().sum()

tp_presenca_cn            0
tp_presenca_ch            0
tp_presenca_lc            0
tp_presenca_mt            0
nu_nota_cn           900794
nu_nota_ch           762323
nu_nota_lc           762323
nu_nota_mt           900794
tp_lingua                 0
tp_status_redacao    762324
nu_nota_redacao      762324
dtype: int64

In [15]:
data[dados_socioeconomico].isna().sum()

q001    0
q002    0
q003    0
q004    0
q005    0
q006    0
q007    0
q008    0
q009    0
q010    0
q011    0
q012    0
q013    0
q014    0
q015    0
q016    0
q017    0
q018    0
q019    0
q020    0
q021    0
q022    0
q023    0
q024    0
q025    0
dtype: int64

**2.3 CHECK DTYPES**

In [18]:
data.dtypes

nu_inscricao                   category
co_municipio_residencia           int32
no_municipio_residencia        category
co_uf_residencia                   int8
sg_uf_residencia               category
nu_idade                        float16
tp_sexo                        category
tp_estado_civil                    int8
tp_cor_raca                        int8
tp_nacionalidade                   int8
co_municipio_nascimento         float32
no_municipio_nascimento        category
co_uf_nascimento                float16
sg_uf_nascimento               category
tp_st_conclusao                    int8
tp_ano_concluiu                    int8
tp_escola                          int8
tp_ensino                       float16
in_treineiro                       int8
co_escola                       float32
co_municipio_esc                float32
no_municipio_esc               category
co_uf_esc                       float16
sg_uf_esc                      category
tp_dependencia_adm_esc          float16


**3.0 DATA FILTERING**

In [16]:
df1 = data.copy()

**3.1 FILLOUT NA**

In [17]:
df1 = df1.dropna(subset=['nu_nota_cn', 'nu_nota_ch', 'nu_nota_lc', 'nu_nota_mt', 'nu_nota_redacao'])

**3.2 ESTATÍSTICA DESCRITIVA**

In [19]:
num_attributes = df1[dados_prova]
cat_attributes = data.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'])

Skew: mostra a deformação da sua distribuição em relação a normal

**Skew positiva:** deformação para a esquerda

**Skew negativa:** deformação para a direita

Kurtosis: mostra o quão os dados estão concentrados.

Quanto maior a kurtosis mais dados concentrados, ou seja o pico da curva será muito grande

Quanto menor a kurtosis,mais dispersos estão os dados, ou seja, o pico da curva será menor

In [21]:
#Central Tendecy - média e mediana
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

#Dispersion - desvio, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

#concat
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min','max','range','mean','median','std','skew','kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,tp_presenca_cn,1.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00
1,tp_presenca_ch,1.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00
2,tp_presenca_lc,1.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00
3,tp_presenca_mt,1.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00
4,nu_nota_cn,0.00,860.90,860.90,477.87,470.30,76.04,0.36,-0.25
5,nu_nota_ch,0.00,835.10,835.10,509.87,513.30,81.02,-0.19,0.41
6,nu_nota_lc,0.00,801.70,801.70,522.39,527.80,62.76,-0.60,1.41
7,nu_nota_mt,0.00,985.50,985.50,523.16,501.10,109.04,0.69,-0.06
8,tp_lingua,0.00,1.00,1.00,0.50,1.00,0.50,-0.02,-2.00
9,tp_status_redacao,1.00,9.00,8.00,NaN,1.00,0.00,NaN,NaN


4.0 EXPLORE DATA ANALYSIS

In [22]:
df2 = df1.copy()